In [151]:
import pandas as pd
import openpyxl
import os
import re
import timeit
import numpy as np
import json
import pytz
import zipfile
from datetime import datetime

## constants

In [152]:
MAX_FILES_TO_PROCESS = 99

INPUT_FOLDER = "./input"
OUTPUT_FOLDER = "./output"

CACHE_FOLDER = "cache"
OUTLOOKS_FOLDER = "outlooks"

TIME_ZONE = pytz.timezone("Australia/Sydney")


## fueltech mappings

In [153]:
FUELTECH_MAPPINGS = {
    "Brown Coal": "coal_brown",
    "Black Coal": "coal_black",
    "Solar Thermal": "solar_thermal",
    "Utility-scale Solar": "solar_utility",
    "Imports": "imports",
    "Exports": "exports",
    "Distributed PV": "solar_rooftop",
    "Wind": "wind",
    "Hydro": "hydro",
    "Distributed Storage": "battery_distributed_discharging",
    "Distributed Storage Load": "battery_distributed_charging",
    "Mid-merit Gas": "gas_ccgt",
    "Mid-merit Gas with CCS": "gas_ccgt_ccs",
    "Offshore Wind": "wind_offshore",
    "Peaking Gas\+Liquids": "gas_ocgt",
    "Hydrogen Turbine": "gas_hydrogen",
    "Utility-scale Storage": "battery_discharging",
    "Utility-scale Storage Load": "battery_charging",
    "Offshore wind": "wind_offshore",
    "Coordinated DER Storage": "battery_VPP_discharging",
    "Coordinated DER Storage Load": "battery_VPP_charging",
    "DSP": "demand_response",

    # 2024 draft
    "Flexible Gas": "gas_ocgt",
    "Passive CER Storage": "battery_distributed_discharging",
    "Passive CER Storage Load": "battery_distributed_charging",
    "Coordinated CER Storage": "battery_VPP_discharging",
    "Coordinated CER Storage Load": "battery_VPP_charging",
    "Biomass": "bioenergy_biomass",

    # 2024 final
    "Brown coal": "coal_brown",
    "Black coal": "coal_black",
    "Mid-merit gas": "gas_ccgt",
    "Flexible gas with CCS": "gas_ccgt_ccs",
    "Flexible gas": "gas_ocgt",

    "Utility storage": "battery_discharging",
    "Utility storage load": "battery_charging",
    "Coordinated CER storage": "battery_VPP_discharging",
    "Coordinated CER storage load": "battery_VPP_charging",
    "Passive CER storage": "battery_distributed_discharging",
    "Passive CER storage load": "battery_distributed_charging",

    "Utility solar": "solar_utility",
    "Other renewable fuels": "bioenergy",
}

# Reverse the FUEL_TECH_MAPPINGS dictionary
FUELTECH_TO_DESCRIPTION = {v: k for k, v in FUELTECH_MAPPINGS.items()}

def get_fuelTechDescription(fuel_tech_id):
    # Use the reversed dictionary to get the readable string
    return FUELTECH_TO_DESCRIPTION.get(fuel_tech_id, fuel_tech_id)

FUELTECH_COLORS = {
    'battery_charging': '#4F5FD7',
    'battery_discharging': '#3145CE',
    'battery_VPP_charging': '#4F5FD7',
    'battery_VPP_discharging': '#3145CE',
    'battery_distributed_charging': '#4F5FD7',
    'battery_distributed_discharging': '#3145CE',
    'battery': '#3145CE',

    'bioenergy': '#069FAF',
    'bioenergy_biogas': '#069FAF',
    'bioenergy_biomass': '#0B757C',

    'coal': '#251C00',
    'coal_black': '#251C00',
    'coal_brown': '#675B42',

    'distillate': '#E46E56',

    'gas': '#E78114',
    'gas_ccgt': '#ED9C2C',
    'gas_ccgt_ccs': '#F1AB4B',
    'gas_ocgt': '#F0AC4A',
    'gas_recip': '#F4C379',
    'gas_steam': '#E78114',
    'gas_wcmg': '#DA630E',
    'gas_hydrogen': '#C75338',

    'hydro': '#ACE9FE',
    'pumps': '#00A5F1',

    'solar': '#FECE00',
    'solar_utility': '#FECE00',
    'solar_thermal': '#FDB200',
    'solar_rooftop': '#FFEB5C',

    'wind': '#246D36',
    'wind_offshore': '#53AD69',

    'nuclear': '#C75338',

    'imports': '#CFA7FF',
    'exports': '#722AF7',
    'interconnector': '#7F7F7F',
    'demand_response': '#7F7F7F',

    'fossil_fuels': '#594929',
    'renewables': '#52A972'
}

## utilities

### unhide sheets

For some reason, AEMO likes to keep most of the interesting sheets in the outlook workbooks hidden. Point this cell at a directory and it'll unhide all the hidden sheets in all the workbooks within.

The unlocked file is written out to the root director of the project. You'll then need to open the workbook in Excel, let it repair the file and save it as a new file.

In [154]:
def unhide_sheets(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            print("\nchecking", filename)
            workbook = openpyxl.load_workbook(os.path.join(directory, filename), keep_links=False)
            print("opened", filename)
    
            for sheet in workbook.sheetnames:
                print("checking", sheet)
                if workbook[sheet].sheet_state == 'hidden':
                    workbook[sheet].sheet_state = 'visible'
            workbook.save(filename)

# unhide_sheets("data")

### print utilities

In [155]:

def countEntriesByTechnology(data_frame, type):
    print("\nCount of type:", type)
    filtered_data_frame = data_frame[data_frame['Type'] == type]
    count_by_technology = filtered_data_frame.pivot_table(index='Technology', columns='CDP', values='Scenario', aggfunc='count', fill_value=0)
    print(count_by_technology)

def printDataSummary(data, startTime):
        countEntriesByTechnology(data, "energy")
        countEntriesByTechnology(data, "capacity")
        elapsed = timeit.default_timer() - startTime
        print(f"rows: {len(data)} processed in {elapsed:.1f} seconds")

## loading from outlooks workbook

### checks

In [156]:
def getYearsFromColumnNames(frame):
    # for col in frame.columns:
    #     print(f">>> col '{col}': {type(col)}")

    # print(frame.columns)
    # frame.info()
    
    years = [col for col in frame.columns if isinstance(col, int)]

    # print(f">>> years: {years}")
    years.sort()
    # print(f">>> years: {years}")

    if (len(years) == 0):
        raise ValueError("ERROR: no columns with years found!")
    
    return years

# check that all technology types in the dataframe are in alphabetical allowing underscrores
# fail hard if they're not
def checkTechnologyTypesAreMapped(combined):
    technology_types = combined['Technology'].unique()

    illegals = [tech for tech in technology_types if not tech.replace('_', '').isalpha()]
    if illegals:
        raise ValueError(f'ERROR: the following technology types are not mapped properly: {illegals}')
    print("INFO: all technology types appear to be mapped properly")


# get a list of all columns with a name that looks like a year, make sure there are at least 20 and they are sequential
def checkYearColumns(combined):
    years = getYearsFromColumnNames(combined)

    if len(years) <= 20 or not all(y2 == y1 + 1 for y1, y2 in zip(years, years[1:])):
        raise ValueError("ERROR: year columns should sequential and in format XXXX (eg. not XXXX-YY).")
    print(f"INFO: years are sequential from {years[0]} to {years[-1]}")


def checkFuelTechValuesLegal(combined):
    # get the list of all columns that look like a year
    years = getYearsFromColumnNames(combined)

    # copy only the rows where any year column has a value less than 0 or NaN
    failed_rows = combined[(combined[years] < 0).any(axis=1) | combined[years].isna().any(axis=1)]

    if not failed_rows.empty:
        # print the number of failed rows
        print(f"ERROR: {len(failed_rows)} of {len(combined)} rows have illegal values")
        print(failed_rows.head())

        # build a list of failed tech types
        failed_tech_types = failed_rows['Technology'].unique()
        raise ValueError(f"ERROR: the following technology types have illegal values: {failed_tech_types}")
    
    print("INFO: all technology types have valid values")


def buildRegionLists(df):
    print("INFO: checking to see that all regions are represented and that no illegal regions are present")

    # Ensure the 'region' column exists in the dataframe
    if 'Region' not in df.columns:
        raise ValueError("ERROR: frame does not have a 'Region' column")
    
    # Define the valid regions
    REGIONS = ['_all', 'qld1', 'nsw1', 'vic1', 'tas1', 'sa1']

    # group by 'Region' and count the rows
    region_counts = df['Region'].value_counts().reset_index()
    region_counts.columns = ['Region', 'count']
    region_list = list(region_counts.itertuples(index=False, name=None))

    # initialise lists
    good_regions = []
    bad_regions = []
    missing_regions = []

    # populate good_regions and bad_regions
    for region, count in region_list:
        if region in REGIONS:
            good_regions.append((region, count))
        else:
            bad_regions.append((region, count))

    # populate missing_regions
    existing_regions = set([region for region, count in region_list])
    missing_regions = [region for region in REGIONS if region not in existing_regions]

    return good_regions, bad_regions, missing_regions


def checkRegions(combined):
    good_regions, bad_regions, missing_regions = buildRegionLists(combined)

    print(f"INFO: got good regions: {good_regions}")

    if (len(missing_regions) > 0):
        raise ValueError(f"ERROR: missing regions: {missing_regions}")
    else:
        print("INFO: all regions are present")

    if (len(bad_regions) > 0):
        raise ValueError(f"ERROR: illegal region names: {bad_regions}")


def runIntegrityChecks(outlooks):
    checkRegions(outlooks)
    checkTechnologyTypesAreMapped(outlooks)
    checkYearColumns(outlooks)
    checkFuelTechValuesLegal(outlooks)

#### tests

In [157]:
import unittest

# Define the unit tests
class TestRegionFunctions(unittest.TestCase):
    def setUp(self):
        # Build the dataframes for testing
        self.good_df = pd.DataFrame({
            'Year': ['2020', '2021', '2022', '2023', '2024', '2025'],
            'Region': ["_all", "nsw1", "sa1", 'vic1', 'tas1', 'qld1']
        })

        self.bad_df = pd.DataFrame({
            'Year': ['2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027'],
            'Region': ["_all", "nsw1", "sa1", 'vic1', 'tas1', 'qld1', 'NSW1', 'foo']
        })
        
        self.missing_df = pd.DataFrame({
            'Year': ['2020', '2021', '2022'],
            'Region': ["_all", "nsw1", "sa1"]
        })

    def test_buildRegionLists_good(self):
        # print("\ntest_buildRegionLists_good")
        # print(self.good_df)

        good, bad, missing = buildRegionLists(self.good_df)
        self.assertEqual(sorted(good), sorted([('_all', 1), ('nsw1', 1), ('sa1', 1), ('vic1', 1), ('tas1', 1), ('qld1', 1)]))
        self.assertEqual(bad, [])
        self.assertEqual(missing, [])

    def test_buildRegionLists_bad(self):
        # print("\ntest_buildRegionLists_bad")
        # print(self.bad_df)
    
        good, bad, missing = buildRegionLists(self.bad_df)
        self.assertEqual(sorted(good), sorted([('_all', 1), ('nsw1', 1), ('sa1', 1), ('vic1', 1), ('tas1', 1), ('qld1', 1)]))
        self.assertEqual(sorted(bad), sorted([('foo', 1), ('NSW1', 1)]))
        self.assertEqual(missing, [])

    def test_buildRegionLists_missing(self):
        # print("\ntest_buildRegionLists_missing")
        # print(self.missing_df)

        good, bad, missing = buildRegionLists(self.missing_df)
        self.assertEqual(sorted(good), sorted([('_all', 1), ('nsw1', 1), ('sa1', 1)]))
        self.assertEqual(bad, [])
        self.assertEqual(sorted(missing), sorted(['qld1', 'vic1', 'tas1']))
    
    def test_checkRegions_all_good(self):
        # print("\ntest_checkRegions_all_good")
        # print(self.good_df)
    
        try:
            checkRegions(self.good_df)
        except ValueError as e:
            self.fail(f"checkRegions raised ValueError unexpectedly: {e}")

    def test_checkRegions_bad_regions(self):
        # print("\ntest_checkRegions_bad_regions")
        # print(self.bad_df)

        with self.assertRaises(ValueError) as context:
            checkRegions(self.bad_df)
        self.assertIn("ERROR: illegal region names:", str(context.exception))

    def test_checkRegions_missing_regions(self):
        # print("\ntest_checkRegions_missing_regions")
        # print(self.missing_df)

        with self.assertRaises(ValueError) as context:
            checkRegions(self.missing_df)
        self.assertIn("ERROR: missing regions:", str(context.exception))


# Run the tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_all_good_regions (__main__.TestBuildRegionLists) ... ok
test_bad_regions (__main__.TestBuildRegionLists) ... ok
test_missing_regions (__main__.TestBuildRegionLists) ... ok
test_buildRegionLists_bad (__main__.TestRegionFunctions) ... ok
test_buildRegionLists_good (__main__.TestRegionFunctions) ... ok
test_buildRegionLists_missing (__main__.TestRegionFunctions) ... ok
test_checkRegions_all_good (__main__.TestRegionFunctions) ... ok
test_checkRegions_bad_regions (__main__.TestRegionFunctions) ... ok
test_checkRegions_missing_regions (__main__.TestRegionFunctions) ... 

INFO: checking to see that all regions are represented and that no illegal regions are present
INFO: checking to see that all regions are represented and that no illegal regions are present
INFO: checking to see that all regions are represented and that no illegal regions are present
INFO: checking to see that all regions are represented and that no illegal regions are present
INFO: checking to see that all regions are represented and that no illegal regions are present
INFO: checking to see that all regions are represented and that no illegal regions are present
INFO: checking to see that all regions are represented and that no illegal regions are present
INFO: got good regions: [('_all', 1), ('nsw1', 1), ('sa1', 1), ('vic1', 1), ('tas1', 1), ('qld1', 1)]
INFO: all regions are present
INFO: checking to see that all regions are represented and that no illegal regions are present
INFO: got good regions: [('_all', 1), ('nsw1', 1), ('sa1', 1), ('vic1', 1), ('tas1', 1), ('qld1', 1)]
INFO: 

ok

----------------------------------------------------------------------
Ran 9 tests in 0.037s

OK


### fixes

In [158]:
def renameRegions(frame):
    # if the region is _all, leave it as is, otherwise shift to lower case and append  '1', eg 'nsw1'
    frame['Region'] = frame['Region'].apply(lambda x: '_all' if x.lower() == "nem" else x.lower() + "1")
    return frame
  

def renameTechnologyLabels(frame):  
    for old_label, new_label in FUELTECH_MAPPINGS.items():
        frame['Technology'] = frame['Technology'].replace(r'^{}$'.format(old_label), new_label, regex=True) 
    return frame


# rename financial year coluumns
# take a data frame, and for each column with a title in the form of '20xx-yy', rename it to 20yy (an integer)
def renameYearColumnsFromStringToInteger(frame):
    print("INFO: renaming year columns from string to integer")
    count = 0

    for col in frame.columns:
        if isinstance(col, str) and re.match(r"^\d{4}$", col):
            frame.rename(columns={col: int(col)}, inplace=True)
            count += 1

    if count:
        print(f"INFO: renamed {count} year columns from string to integer")


# rename financial year coluumns
# take a data frame, and for each column with a title in the form of '20xx-yy', rename it to 20yy (an integer)
def renameFinancialYearColumns(frame):
    print("INFO: renaming financial year columns…")
    count = 0

    for col in frame.columns:
        if isinstance(col, str) and re.match(r"^\d{4}-\d{2}$", col):
            new_col_name = int("20" + col[5:])
            frame.rename(columns={col: new_col_name}, inplace=True)
            count += 1

    if count:
        print(f"INFO: renamed {count} financial years")


def makeSpecialTechsPositive(frame):
    # print("INFO: flipping negative values")

    # get the list of all columns that look like a year
    years = getYearsFromColumnNames(frame)

    flip_types = [
        "battery_charging",
        "battery_VPP_charging",
        "battery_distributed_charging",
        "exports",
    ]

    for tech_type in flip_types:
        # pull out the rows with column Technology = tech_type
        tech_rows = frame[frame["Technology"] == tech_type]
        
        # if any of the years are negative for the given tech_type
        num_negative = tech_rows[(tech_rows[years] < 0).any(axis=1)].shape[0]
        
        if num_negative:
            # …make them positive
            print(f"INFO: {num_negative} rows with negative values found for '{tech_type}', making them positive")
            frame.loc[frame['Technology'] == tech_type, years] = frame.loc[frame['Technology'] == tech_type, years].abs()


def changeNumericColumnsToFloats(frame):
    # for all columns with a year header (format yyyy), chnage the type to a float
    for col in getYearsFromColumnNames(frame):
        frame[col] = frame[col].astype(float)


def multiplyBy1e6(frame):
    # for all columns with a year header, multiply values by 1e6
    for col in getYearsFromColumnNames(frame):
        frame[col] = frame[col] * 1e6


def collapseSubregions(frame):
    # check whether the dataframe has a column called Subregion
    if 'Subregion' not in frame.columns:
        # print("INFO: subregion column not found, no collapsing needed")
        return frame
    
    print("INFO: subregion column found, collapsing")
    before_rows = frame.shape[0]

    # make a new dataframe grouping by CDP, Region, Technology
    collapsed = frame.groupby(['CDP', 'Region', 'Technology']).sum().reset_index()

    # drop the Subregion column
    collapsed = collapsed.drop(columns=['Subregion'])

    after_rows = collapsed.shape[0]

    print(f"INFO: collapsing subregions went from {before_rows} to {after_rows} rows")
    return collapsed


def addSummaryRegion(df):
    # create an _all region, being the sum of all regions within each scenario, CDP, type and technology
    nem_region = df.groupby(["Scenario", "CDP", "Type", "Technology"]).sum().reset_index()
    nem_region["Region"] = "_all"

    # exclude entries where Technology is 'imports' or 'exports'
    trim_nem = nem_region[~nem_region['Technology'].isin(['imports', 'exports'])]

    return pd.concat([df, trim_nem], ignore_index=True)

### test flip

In [159]:
# generate some demo data to feed into flipNegativeValues
def getDemoData():
    data = {
        'Technology': ['battery_charging', 'exports', 'solar_utility'],
        '2019': ["-10.5", "-20.5", "-30.4"],
        '2020': ["-5", "-15", "-25.1"],
        '2021': ["-7", "-17", "-27.5",]
    }
    return pd.DataFrame(data)

def testFlip():
    # Generate demo data
    demo = getDemoData()
    demo.info()
    print(demo)
    changeNumericColumnsToFloats(demo)
    demo.info()
    print(demo)
    makeSpecialTechsPositive(demo)
    print(demo)

# testFlip()

### loading and caching

In [160]:
def loadISPDataFromSheet(excel_file, sheetname):
    sheet = excel_file[sheetname]
    data = pd.DataFrame(sheet.values)
    
    # grab the column names from the 3rd row, using integers for the year headings
    data.columns = data.iloc[2].apply(lambda x: int(float(x)) if isinstance(x, float) else x)

    # remove blank rows
    data.drop([0, 1, 2], inplace=True)  
    data.dropna(how="all", inplace=True)

    return data


def getWorkbookData(release_id, file_name, label):
    workbook_path = os.path.join(INPUT_FOLDER, release_id, file_name)
    print(f"\nloading release '{release_id}', scenario '{label}' from '{workbook_path}'")

    excel_file = openpyxl.load_workbook(workbook_path)
  
    # grab generation, using it as the basic structure
    print("INFO: loading generation")
    generation = loadISPDataFromSheet(excel_file, 'Generation')

    # grab the flows
    print("INFO: loading imports and exports")
    flows = loadISPDataFromSheet(excel_file, 'Imports and Exports')
    
    if 'Flow' in flows.columns:  # ensure that 'Flow' column exists
        flows = flows[flows['Flow'].isin(['Imports', 'Exports'])]
        flows = flows.rename(columns={"Flow": "Technology"})
      
        # flip the sign of numbers in the row for exports
        flows.loc[flows['Technology'] == "Exports", generation.select_dtypes(include=[np.number]).columns] *= -1
    else:
        print("Column names of flows:", list(flows.columns))
        raise ValueError("ERROR: the Flows column is missing")
      
    # grab the capacities 
    print("INFO: loading capacity")
    capacities = loadISPDataFromSheet(excel_file, 'Capacity')

    # grab the emissions
    print("INFO: loading emissions")
    emissions = loadISPDataFromSheet(excel_file, 'Emissions')
    emissions = emissions.drop(columns=['Total'])       # remove the Total column
    emissions['Technology'] = "none"                    # set the Technology column to "none" 
    
    # remove the column 'Existing and Committed' 
    if (release_id == '2022_ISP_final'):
        print("INFO: removing column 'Existing and Committed' from capacities")
        capacities.drop(columns=['Existing and Committed'], inplace=True)

    # 2024 draft
    # TODO only drop for 2024 draft?
    #capacities = capacities.drop(2024, axis=1)

    # 2024 final: remove '2023-24' from capacities
    if (release_id == '2024_ISP_final'):
        capacities.drop(columns=['2023-24'], inplace=True)

    # some workbooks have year columns in format 2023-24, others in 2024, so rename them all to 2024
    dataframes = [capacities, generation, flows, emissions]
    for frame in dataframes:
        renameFinancialYearColumns(frame)
        changeNumericColumnsToFloats(frame)
        renameRegions(frame)

    # introduced for 2024 final: remove 'Subregion' column if it exists
    capacities = collapseSubregions(capacities)
    generation = collapseSubregions(generation)

    # check the dataframes are compatible 
    if set(generation.columns) != set(flows.columns) or set(generation.columns) != set(capacities.columns) or set(generation.columns) != set(emissions.columns):
        print("generation columns:", list(generation.columns))
        print("flows columns:", list(flows.columns))
        print("capacities columns:", list(capacities.columns))
        print("emissions columns:", list(emissions.columns))
        raise ValueError("ERROR: The column names of generation, flows, capacities and emissions are not identical.")

    # touch up the energyies
    energies = pd.concat([generation, flows], ignore_index=True)
    energies.insert(0, "Type", "energy")
    energies = renameTechnologyLabels(energies)
    makeSpecialTechsPositive(energies)                   # flip the sign of negative values
  
    # touch up the capacities
    capacities.insert(1, "Type", "capacity")
    capacities = renameTechnologyLabels(capacities)
    capacities['Technology'] = capacities['Technology'].apply(lambda x: x[:-12] if x.endswith("_discharging") else x)  # remove _discharging from tech labels
    
    # touch up the emissions
    emissions.insert(1, "Type", "emissions")
    multiplyBy1e6(emissions)

    # now combine them and add a column with the file label (in snake case)
    combined = pd.concat([energies, capacities, emissions], ignore_index=True)
    combined.insert(0, "Scenario", re.sub(r'\W+', '_', label.strip().lower()))

    combined = addSummaryRegion(combined)
  
    return combined


def loadScenariosList(release_name):
    # load the list of ISP files from the scenarios JSON file
    index_file = os.path.join(INPUT_FOLDER, release_name, "scenarios.json")
    if not os.path.exists(index_file):
        raise ValueError(f"ERROR: the scenario index file '{index_file}' does not exist.")
    
    with open(index_file) as f:
        scenario_files = json.load(f)    

    if len(scenario_files) == 0:
        raise ValueError(f"ERROR: the scenarios list from '{index_file}' is empty.")
    
    return scenario_files


def processGenerationOutlookFiles(release_id):
    # process_start_time = timeit.default_timer()
    combined_data = pd.DataFrame()

    num_files_processed = 0

    scenario_files = loadScenariosList(release_id)
    for file_info in scenario_files:
        if num_files_processed >= MAX_FILES_TO_PROCESS:
            break
      
        # file_start_time = timeit.default_timer()
        scenario_label = file_info["label"]
        file_name = file_info['file_name']
        outlook_data = getWorkbookData(release_id, file_name, scenario_label)
        # printDataSummary(outlook_data, file_start_time)
        
        combined_data = pd.concat([combined_data, outlook_data], ignore_index=True)
        num_files_processed += 1

    # print final summary and write CSV file
    # print("\n\naggregated data")
    # printDataSummary(combined_data, process_start_time)

    return combined_data


def processAndCacheOutlooks(filename_parquet, release_name):
    if os.path.exists(filename_parquet):
        print(f"WARNING: '{filename_parquet}' exists, skipping processing and will used cached version")
        return

    print(f"\nINFO: processing ISP outlook workbooks for release '{release_name}'")
    combined_data = processGenerationOutlookFiles(release_name)

    # # output as a CSV file in the cache folder
    # FILENAME_CSV = os.path.join(cache_path, release_name + ".outlook.csv")
    # combined_data.to_csv(FILENAME_CSV)

    runIntegrityChecks(combined_data)

    # convert all column headers to strings
    print(f"\nwriting to cache '{filename_parquet}'")
    frame_copy = combined_data.copy()
    frame_copy.columns = frame_copy.columns.map(str)
    frame_copy.to_parquet(filename_parquet)


# read from cache if it exists, otherwise process and write to cache
def loadGenerationOutlooks(release_name):
    # compute the cache director name and ensure that it exists
    cache_path = os.path.join(OUTPUT_FOLDER, CACHE_FOLDER)
    if not os.path.exists(cache_path):
        print("creating cache directory")
        os.makedirs(cache_path)

    filename_parquet = os.path.join(cache_path, release_name + ".outlook.parquet")

    # process and cache
    processAndCacheOutlooks(filename_parquet, release_name)

    # load from cache
    combined_data = pd.read_parquet(filename_parquet)
    renameYearColumnsFromStringToInteger(combined_data)
    runIntegrityChecks(combined_data)
    
    return combined_data


# outlooks = loadGenerationOutlooks(RELEASE)

## build a simple chart

In [161]:
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'  # For high-resolution displays

def reorder_columns(data):
    FUELTECH_ORDER = [
        'battery_charging',
        'battery_VPP_discharging',
        'battery_distributed_discharging',
        'pumps',
        'exports',
        'coal_brown',
        'coal_black',
        'bioenergy',
        'bioenergy_biogas',
        'bioenergy_biomass',
        'distillate',
        'gas_steam',
        'gas_ccgt',
        'gas_ocgt',
        'gas_recip',
        'gas_wcmg',
        'gas_ccgt_ccs',
        'gas_ocgt_ccs',
        'gas_hydrogen',
        'battery_discharging',
        'hydro',
        'wind_offshore',
        'wind',
        'solar_thermal',
        'solar_utility',
        'solar_rooftop',
        'demand_response'
    ]

    # battery_VPP_discharging
    # battery_distributed_discharging
    # demand_response
    # gas_ccgt_ccs
    # gas_hydrogen
    # solar_thermal
    # wind_offshore

    # Get the intersection of the order and the columns in the data
    common_columns = [col for col in FUELTECH_ORDER if col in data.columns]

    # Check if there are any columns in the data that are not in the order
    extra_columns = [col for col in data.columns if col not in FUELTECH_ORDER]
    if extra_columns:
        print(f"WARN: data contains the following fuel techs not found in FUEL_TECH_ORDER: {', '.join(extra_columns)}")

    # Reorder the columns in data
    reordered_data = data[common_columns]
    return reordered_data


def create_area_chart(outlooks, scenario, type, CDP, region):
    # Filter the data based on the provided parameters
    filtered_data = outlooks[(outlooks['Scenario'] == scenario) & (outlooks['Type'] == type) & (outlooks['CDP'] == CDP) & (outlooks['Region'] == region)]

    # Reshape the data so that each year is a separate row
    reshaped_data = filtered_data.melt(id_vars=['Scenario', 'Type', 'CDP', 'Region', 'Technology'], var_name='Year', value_name='Value')

    # Pivot the data so that each technology is a separate column
    pivoted_data = reshaped_data.pivot(index='Year', columns='Technology', values='Value')

    # Remove columns ending with '_charging'
    pivoted_data = pivoted_data[pivoted_data.columns.drop(list(pivoted_data.filter(regex='_charging')))]

    # Divide all values in the DataFrame by 1000
    pivoted_data = pivoted_data.divide(1000)

    # Reorder the columns in pivoted_data
    pivoted_data = reorder_columns(pivoted_data)

    # Create a new figure with a specific size
    plt.figure(figsize=(10, 5))

    # Create a list of colors for the columns in pivoted_data
    colors = [FUELTECH_COLORS.get(fueltech_id, '#000000') for fueltech_id in pivoted_data.columns]

    # Plot the stacked area chart
    plt.stackplot(pivoted_data.index, pivoted_data.T, colors=colors)

    # Rotate x-axis labels 90 degrees
    plt.xticks(rotation=90)

    plt.ylabel('TWh')
    plt.title(f'Generation — {scenario}/{region}/{CDP}', fontname='Arial Black', fontsize=20)

    # Map the column names to their display names
    display_names = [get_fuelTechDescription(fueltech_id) for fueltech_id in pivoted_data.columns]

    # Move the legend to the right of the chart
    plt.legend(display_names, bbox_to_anchor=(1.00, 1), loc='upper left', frameon=False)

    # Remove the right and top spines
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['left'].set_color('lightgrey')    
    plt.gca().spines['bottom'].set_color('lightgrey')
    
    # Add x-axis and y-axis in light grey
    plt.axhline(0, color='grey')

    # Make all axes and tick marks in light grey
    plt.tick_params(colors='grey')

    plt.show()

# create_area_chart(outlooks, 'step_change', 'energy', 'CDP3', 'nem')


# TESTING

### testing collapsing subregions

In [162]:
# show all the entries for CDP=CDP1 in Region=NSW1 and Technology=coal_black
def printCDP1_NSW1_Coal(frame):
    print(frame[(frame['CDP'] == 'CDP1') & (frame['Region'] == 'NSW1') & (frame['Technology'] == 'coal_black')])


def myGenTest():
    # load the generation parquet file
    energies = pd.read_parquet("generation.parquet")
    renameTechnologyLabels(energies)
    renameRegions(energies)
    makeSpecialTechsPositive(energies)

    print("\n\n>>> before:")
    printCDP1_NSW1_Coal(energies)

    # make a new dataframe with all columns except the Subregion column, grouping by CDP, Region, Technology
    new_energies = collapseSubregions(energies)

    print("\n\n>>> after:")
    printCDP1_NSW1_Coal(new_energies)

    new_energies.info()
    print(new_energies.head())

    return new_energies

# my_gen = myGenTest()
# my_gen

## output to JSON

### zip utilities

In [163]:
def zipdir(path, zip_path):
    zipf = zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED)
    for root, dirs, files in os.walk(path):
      for file in files:
        zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))
    zipf.close()

    # zipdir('outlooks', 'outlooks.zip')

### JSON utilities

In [164]:
# put all the elements of the numerical array on the same line
def compact_json(json_str):
    # This regex looks for the _numerical_ data array and captures everything in between [ and ]
    pattern = r'("data": \[)([\d\., \n]*)(\])'
    
    # The inner function is used to replace newlines and spaces between array items
    def replacer(match):
        start, middle, end = match.groups()
        # Replace newline and possible spaces
        compact_middle = middle.replace('\n', '').replace(' ', '').replace(',', ', ')
        return start + compact_middle + end
    
    return re.sub(pattern, replacer, json_str, flags=re.DOTALL)

def generate_id(row):
  if row['Region'] == 'nem':
    id = "au.nem."
  else:
    id = f"au.nem.{row['Region'].lower()}."

  if row['Technology'] != 'none':
      id += f"fuel_tech.{row['Technology']}."
  
  return id + f"{row['Type']}.{row['Scenario']}.{row['CDP'].lower()}"

## generate JSON

In [165]:
RELEASES_FOLDER = "releases"

def getUnitFromType(series_type):
    if series_type == "energy":
        return "GWh"
    elif series_type == "capacity":
        return "MW"
    elif series_type == "emissions":
        return "tCO2e"
    else:
        raise ValueError(f"ERROR: unknown type '{series_type}'")

def buildNewJSON(outlooks, release, scenario):
    data = []

    # extract start and end year
    years = getYearsFromColumnNames(outlooks)
    start_year = years[0] - 1   # subtract 1 to account for start of financial year
    last_year = years[-1] - 1
    start = f"{start_year}-07-01T00:00:00+10:00"
    last = f"{last_year}-07-01T00:00:00+10:00"
   
    for i, row in outlooks.iterrows():
        if row['Scenario'] == scenario:
            # change numbers to just 1 decimal place
            year_data = [round(x, 1) for x in row[pd.to_numeric(row.index, errors='coerce')>=2022].tolist()]  # selecting only year data
              
            element = {
                "id": generate_id(row),
                "type": row['Type'],
                "network": "nem",
                "region": row['Region'],
                "fuel_tech": row['Technology'],
                "scenario": row['Scenario'],
                "pathway": row['CDP'],
                "units": getUnitFromType(row['Type']),
                "projection": {
                    "start": start,
                    "last": last,
                    "interval": "1Y",
                    "data": year_data
                }
            }
              
            data.append(element)

    output_obj = {
        "version": "4.1",
        "release": release,
        "created_at": datetime.now(TIME_ZONE).strftime("%Y-%m-%dT%H:%M:%S%z"),
        "messages": [ f"projections from AEMO's {release} generation outlooks" ],
        "data": data,
    }

    json_output = json.dumps(output_obj, indent=2)
    return compact_json(json_output)


def writeNewJSON(root, outlooks, release, scenario):
    print(f"INFO: writing {release}/{scenario}")

    json_output = buildNewJSON(outlooks, release, scenario)
    filename = f"{scenario}.json"
    path = os.path.join(root, filename)
    with open(path, "w") as f:
        f.write(json_output)


def writeNewJSONs(release):
    outlooks = loadGenerationOutlooks(release)

    # make the enclosing folder
    root = os.path.join(OUTPUT_FOLDER, RELEASES_FOLDER, release)
    print(f"INFO: creating folder {root}")
    os.makedirs(root, exist_ok=True)

    for scenario in outlooks['Scenario'].unique():
        writeNewJSON(root, outlooks, release, scenario)


writeNewJSONs("2022_ISP_final")
writeNewJSONs("2024_ISP_final")



creating cache directory

INFO: processing ISP outlook workbooks for release '2022_ISP_final'

loading release '2022_ISP_final', scenario 'Step Change' from './input/2022_ISP_final/2022 Final ISP results workbook - Step Change - Updated Inputs.xlsx'
INFO: loading generation
INFO: loading imports and exports
INFO: loading capacity
INFO: loading emissions
INFO: removing column 'Existing and Committed' from capacities
INFO: renaming financial year columns…
INFO: renaming financial year columns…
INFO: renaming financial year columns…
INFO: renaming financial year columns…
INFO: subregion column not found, no collapsing needed
INFO: subregion column not found, no collapsing needed
INFO: flipping negative values
INFO: 50 rows with negative values found for 'battery_charging', making them positive
INFO: 50 rows with negative values found for 'battery_VPP_charging', making them positive
INFO: 50 rows with negative values found for 'battery_distributed_charging', making them positive

loading r